In [1]:
import os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import math
import cv2
from PIL import Image
  

print('Tensorflow Version:', tf.__version__)
print('Keras Version:', keras.__version__)

In [2]:
tf.config.list_physical_devices()

In [3]:
image_root = '../input/car-object-detection/data/training_images'
BATCH_SIZE = 16
IMAGE_SIZE = (380, 676)
MODEL_PATH = './model'

In [4]:
df = pd.read_csv('../input/car-object-detection/data/train_solution_bounding_boxes (1).csv')
df.sample(4)

In [5]:
os.path.join(image_root,'vid_4_6440.jpg')

In [6]:
def draw_box(img, xy):
    x1, y1, x2, y2 = xy.astype('int16')
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 5)
    return img

In [7]:
plt.figure(figsize=(20, 12))

for i, (index, row) in enumerate(df.head(4).iterrows()):
    plt.subplot(2, 2, i+1)
    
    img = Image.open(os.path.join(image_root,row['image']))
    img = np.asarray(img)
#     img = img-img.min()
    img = img/255
    print(img.shape)
    
    img = draw_box(img, np.array([row['xmin'], row['ymin'], row['xmax'], row['ymax']]))
#     plt.title(str(row))
    plt.imshow(img)

# Creating DataSet

In [8]:
# step 1
filenames = tf.constant(df['image'].map(lambda x: os.path.join(image_root,x)))
labels = tf.constant(df[['xmin', 'ymin', 'xmax', 'ymax']])

In [9]:
#step 2
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

In [10]:
for x, y in dataset.take(4):
    print(x)
    print(y)
    print()

In [11]:
def load_img(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32)
    image = keras.applications.xception.preprocess_input(image)
#     print(type(label), label)
    return image, label

dataset = dataset.map(load_img).batch(BATCH_SIZE)

In [12]:
plt.figure(figsize=(20, 12))


for x_batch, y_batch in dataset.take(1):
    height = int(math.sqrt(BATCH_SIZE))
    for i, (image, label) in enumerate(zip(x_batch, y_batch)):
        plt.subplot(height, height, i+1)
        
        img = image.numpy() # [-1, 1]
#         print(img.min(), img.max())
#         img = img-img.min()
#         img = img/img.max()
        img = img+1 # [0, 2]
        img = img/2 # [0, 1]
        
        img = draw_box(img, np.array([*label]))
        
        plt.imshow(img)
        plt.title(img.shape)

plt.show()

In [13]:
# Object Detection = classification + localization on single object

base_model = keras.applications.xception.Xception(weights="imagenet",include_top=False)

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
n_classes = 5
class_output = keras.layers.Dense(n_classes, activation="softmax")(avg) # classification
loc_output = keras.layers.Dense(4)(avg) # localization

model = keras.models.Model(inputs=base_model.input,outputs=[class_output, loc_output])

model.compile(loss=["sparse_categorical_crossentropy", "mse"],loss_weights=[0.8, 0.2],
              optimizer='adam', metrics=[tf.keras.metrics.MeanIoU(num_classes=n_classes)])

In [14]:
# only of single object localization

base_model = keras.applications.xception.Xception(weights="imagenet",include_top=False)

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
loc_output = keras.layers.Dense(4)(avg) # localization

model = keras.models.Model(inputs=base_model.input,outputs=[loc_output])

model.compile(loss=["mse"],
              optimizer='adam')

In [15]:
base_model.trainable = False

In [18]:
val_dataset = dataset.take(4) # 16*4
train_dataset = dataset.skip(4)

In [19]:
early_stopping_cb = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    min_delta=500,
    restore_best_weights=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint(MODEL_PATH)

model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200,
    callbacks=[early_stopping_cb, checkpoint_cb])

In [20]:
model.trainable = True

optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(loss=["mse"],
              optimizer=optimizer
             )

early_stopping_cb = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    min_delta=500,
    restore_best_weights=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint(MODEL_PATH)

model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=[early_stopping_cb, checkpoint_cb])

In [21]:
plt.figure(figsize=(15, 8))

for x_batch, y_batch in val_dataset.take(3):
    for i in range(6):
        plt.subplot(2, 3, i+1)
        
        image = x_batch[i]
        img = image.numpy()
        label = y_batch[i]
        
        img = img+1
        img = img/2
        xy = model(image[tf.newaxis, :]).numpy()[0]
        plt.title("{}".format(xy))
        
        img = draw_box(img, xy)
        
        plt.imshow(img)
        

plt.show()